In [ ]:
import requests
import zipfile,io
import os
import shutil
import pandas as pd
from pandas import *
import graphlab


graphlab.product_key.set_product_key('60B1-D685-6EA3-DD15-4246-4BF7-D25C-CA90')


movietrain_data = pd.read_csv("moviestrain.csv")
movietest_data = pd.read_csv("moviestest.csv")

test = pd.read_csv("testratings.csv")
train = pd.read_csv("trainratings.csv")

test_data = graphlab.SFrame(test)
train_data = graphlab.SFrame(train)


movietrain_data = graphlab.SFrame(movietrain_data)
movietest_data = graphlab.SFrame(movietest_data)

In [14]:
popularity_model = graphlab.popularity_recommender.create(train_data, 
            user_id='userId', item_id='movieId', target='rating')

Recsys training: model = popularity

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 37898 observations with 665 users and 6734 items.

Data prepared in: 0.07664s

37898 observations to process; with 6734 unique items.

In [15]:
popularity_recomm = popularity_model.recommend(users=range(1,6),k=5)

In [16]:
popularity_recomm.print_rows(num_rows=25)

+--------+---------+-------+------+
| userId | movieId | score | rank |
+--------+---------+-------+------+
|   1    |   3873  |  3.0  |  1   |
|   1    |   538   |  3.0  |  2   |
|   1    |    26   |  3.0  |  3   |
|   1    |   3016  |  3.0  |  4   |
|   1    |   2659  |  3.0  |  5   |
|   2    |   3873  |  3.0  |  1   |
|   2    |   538   |  3.0  |  2   |
|   2    |    26   |  3.0  |  3   |
|   2    |   3016  |  3.0  |  4   |
|   2    |   2659  |  3.0  |  5   |
|   3    |   3873  |  3.0  |  1   |
|   3    |   538   |  3.0  |  2   |
|   3    |    26   |  3.0  |  3   |
|   3    |   3016  |  3.0  |  4   |
|   3    |   2659  |  3.0  |  5   |
|   4    |   2038  |  3.0  |  1   |
|   4    |   3885  |  3.0  |  2   |
|   4    |   3873  |  3.0  |  3   |
|   4    |   538   |  3.0  |  4   |
|   4    |    26   |  3.0  |  5   |
|   5    |   3873  |  3.0  |  1   |
|   5    |   538   |  3.0  |  2   |
|   5    |    26   |  3.0  |  3   |
|   5    |   3016  |  3.0  |  4   |
|   5    |   2659  |  3.0  |

In [17]:
item_sim_model = graphlab.item_similarity_recommender.create(train_data, 
user_id='userId', item_id='movieId', target='rating', similarity_type='cosine')

Recsys training: model = item_similarity

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 37898 observations with 665 users and 6734 items.

...

| 2.47s                               | 100              | 6734            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.54008s

In [18]:
item_sim_recomm = item_sim_model.recommend(users=range(600,615),k=25)
item_sim_recomm.print_rows(num_rows=200)

+--------+---------+-----------------+------+
| userId | movieId |      score      | rank |
+--------+---------+-----------------+------+
|  600   |   3276  |  0.256676083803 |  1   |
|  600   |   3596  |  0.256676083803 |  2   |
|  600   |   2887  |  0.256676083803 |  3   |
|  600   |   3031  |  0.256676083803 |  4   |
|  600   |   1022  |  0.245341426134 |  5   |
|  600   |  47382  |  0.220471179485 |  6   |
|  600   |  31193  |  0.220471179485 |  7   |
|  600   |  58105  |  0.21992225647  |  8   |
|  600   |  57326  |  0.215417468548 |  9   |
|  600   |  44193  |  0.214843547344 |  10  |
|  600   |   2090  |  0.212899321318 |  11  |
|  600   |   7102  |  0.20945289731  |  12  |
|  600   |  32882  |  0.207478058338 |  13  |
|  600   |  78517  |  0.207446944714 |  14  |
|  600   |   7334  |  0.207446944714 |  15  |
|  600   |   6584  |  0.207446944714 |  16  |
|  600   |   3334  |  0.207446944714 |  17  |
|  600   |   4628  |  0.205711913109 |  18  |
|  600   |  48982  |  0.2055073142

In [33]:
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.00596125186289 | 3.50364711219e-05 |
|   2    | 0.00372578241431 | 3.94326745606e-05 |
|   3    | 0.00298062593145 | 5.93035141036e-05 |
|   4    | 0.00409836065574 | 7.91492100031e-05 |
|   5    | 0.00387481371088 | 9.17760633001e-05 |
|   6    | 0.00347739692002 | 9.32429067703e-05 |
|   7    | 0.00425803704492 | 0.000132228933206 |
|   8    | 0.00372578241431 | 0.000132228933206 |
|   9    | 0.00380857757907 | 0.000154566052767 |
|   10   | 0.00342771982116 | 0.000154566052767 |
+--------+------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+------------------+
| cutoff | mean_precision |   mean_recall    |
+--------+---------------

### This shows that item similarity model gives better precision and recall.